In [1]:
import deepspeed
from parler_tts import ParlerTTSForConditionalGeneration
from transformers import AutoTokenizer
import soundfile as sf
import torch
import time

[2025-03-11 09:46:58,367] [INFO] [real_accelerator.py:222:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/ece/anaconda3/envs/tf-gpu-211/bin/../lib/gcc/x86_64-conda-linux-gnu/11.2.0/../../../../x86_64-conda-linux-gnu/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/home/ece/anaconda3/envs/tf-gpu-211/bin/../lib/gcc/x86_64-conda-linux-gnu/11.2.0/../../../../x86_64-conda-linux-gnu/bin/ld: cannot find -lcufile: No such file or directory
collect2: error: ld returned 1 exit status
Flash attention 2 is not installed


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_id = "parler-tts/parler_tts_mini_v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = ParlerTTSForConditionalGeneration.from_pretrained(model_id).to(device)

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers
/home/system4/anaconda3/envs/tf-gpu-211/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
Config of the text_encoder: <class 'transformers.models.t5.modeling_t5.T5EncoderModel'> is overwritten by shared text_encoder config: T5Config {
  "_name_or_path": "google/flan-t5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "classifier_dropout": 0.0,
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
 

In [3]:
prompts = [
    "Hello world.",
    "A futuristic AI assistant responding in a clear, robotic yet friendly manner",
    "A dramatic voice-over for an action-packed movie trailer that builds suspense",
    "A casual and friendly conversation starter that feels natural and engaging",
    "An enchanting introduction to a bedtime story that sparks imagination",
    "A detailed weather update that provides temperature, wind speed, and overall forecast",
    "A robotic voice delivering a monotone yet precise system status update",
    "A powerful and energetic motivational speech that inspires action and confidence"
]

descriptions = [
    "A female speaker delivers a warm and welcoming message with a slightly expressive and friendly tone. The speech has a moderate pace and a natural intonation, making it feel inviting. The recording is clear, with minimal background noise.",
    "A robotic AI voice speaks in a neutral yet slightly friendly manner. The speech is steady, with a consistent pitch and minimal variation in tone. The audio is crisp, resembling a synthesized assistant's response.",
    "A deep male voice narrates with a dramatic and intense tone, building suspense with pauses and rising intensity. The speech is slow-paced, emphasizing key moments. The recording is cinematic, with a slight reverberation for a grand effect.",
    "A young adult male speaker talks in a casual and relaxed tone. The speech is natural, with slight variations in pitch and pauses that mimic real-life conversations. The recording is high quality, making it feel like a personal chat.",
    "A soft-spoken female speaker introduces a bedtime story with a soothing and melodic tone. The pace is slow and gentle, making it easy to follow. The recording has a slight warmth, resembling a close-up microphone capture.",
    "A professional male voice delivers a clear and informative weather update. The tone is neutral but engaging, with a moderate pace. The articulation is precise, and the recording is high quality, with no distortions.",
    "A synthetic robotic voice speaks in a completely monotone and even-paced manner. The pitch remains constant, with no emotional inflection. The recording is crisp and clean, resembling an automated system response.",
    "A dynamic and energetic male speaker delivers an inspiring speech with a strong, enthusiastic tone. The pace is varied, with emphasis on key words to motivate the listener. The recording is sharp and immersive, with no background noise."
]

In [4]:
input_ids = tokenizer(descriptions, return_tensors="pt",padding="max_length",max_length=70).to(device)
prompt_input_ids = tokenizer(prompts, return_tensors="pt",padding="max_length",max_length=70).to(device)

: 

## Normal Model Inference

In [7]:
times = []
for i in range(5):
    start_time = time.time()
    with torch.no_grad():
        generation = model.generate(
            input_ids=input_ids.input_ids,
            attention_mask=input_ids.attention_mask,
            prompt_input_ids=prompt_input_ids.input_ids,
            prompt_attention_mask=prompt_input_ids.attention_mask,
        )
    end_time = time.time()
    execution_time = end_time - start_time
    times.append(execution_time)
    print(f"Iteration {i+1} took {execution_time:.2f} seconds")

avg_time = sum(times) / len(times)
print(f"\nAverage execution time: {avg_time:.2f} seconds")

Iteration 1 took 18.21 seconds
Iteration 2 took 19.98 seconds
Iteration 3 took 33.39 seconds
Iteration 4 took 16.33 seconds
Iteration 5 took 14.87 seconds

Average execution time: 20.56 seconds


## Latency Optimization - 1 (Using Enable CUDA Graphs and float16 with caching)

In [8]:
ds_config = {
    "tensor_parallel": {
        "tp_size": 1
    },
    "dtype": torch.float16,
    "replace_with_kernel_inject": True,
    "enable_cuda_graph": True,}


In [9]:
engine = deepspeed.init_inference(
    model=model,
    config=ds_config,
    dtype=torch.float16
)

[2025-03-10 19:35:31,559] [INFO] [logging.py:128:log_dist] [Rank -1] DeepSpeed info: version=0.16.4, git-hash=unknown, git-branch=unknown
[2025-03-10 19:35:31,563] [INFO] [logging.py:128:log_dist] [Rank -1] quantize_bits = 8 mlp_extra_grouping = False, quantize_groups = 1


In [11]:
import time

times = []
for i in range(5):
    start_time = time.time()
    with torch.no_grad():
        generation = engine.module.generate(
            input_ids=input_ids.input_ids,
            attention_mask=input_ids.attention_mask,
            prompt_input_ids=prompt_input_ids.input_ids,
            prompt_attention_mask=prompt_input_ids.attention_mask,
            do_sample=True,
            use_cache=True,
            return_dict_in_generate=True
        )
    end_time = time.time()
    execution_time = end_time - start_time
    times.append(execution_time)
    print(f"Iteration {i+1} took {execution_time:.2f} seconds")

avg_time = sum(times) / len(times)
print(f"\nAverage execution time: {avg_time:.2f} seconds")

Iteration 1 took 16.62 seconds
Iteration 2 took 17.27 seconds
Iteration 3 took 17.99 seconds
Iteration 4 took 19.84 seconds
Iteration 5 took 18.74 seconds

Average execution time: 18.09 seconds


## Latency Optimization - 2 (Using Enable CUDA Graphs and float32 with caching)

In [15]:
ds_config = {
    "tensor_parallel": {
        "tp_size": 1
    },
    "replace_with_kernel_inject": True,
    "enable_cuda_graph": True,}


In [16]:
engine = deepspeed.init_inference(
    model=model,
    config=ds_config,
    dtype=torch.float32
)

[2025-03-10 19:43:08,468] [INFO] [logging.py:128:log_dist] [Rank -1] DeepSpeed info: version=0.16.4, git-hash=unknown, git-branch=unknown
[2025-03-10 19:43:08,472] [INFO] [logging.py:128:log_dist] [Rank -1] quantize_bits = 8 mlp_extra_grouping = False, quantize_groups = 1


In [17]:
import time

times = []
for i in range(5):
    start_time = time.time()
    with torch.no_grad():
        generation = engine.module.generate(
            input_ids=input_ids.input_ids,
            attention_mask=input_ids.attention_mask,
            prompt_input_ids=prompt_input_ids.input_ids,
            prompt_attention_mask=prompt_input_ids.attention_mask,
            do_sample=True,
            use_cache=True,
            return_dict_in_generate=True
        )
    end_time = time.time()
    execution_time = end_time - start_time
    times.append(execution_time)
    print(f"Iteration {i+1} took {execution_time:.2f} seconds")

avg_time = sum(times) / len(times)
print(f"\nAverage execution time: {avg_time:.2f} seconds")

Iteration 1 took 21.40 seconds
Iteration 2 took 18.71 seconds
Iteration 3 took 24.90 seconds
Iteration 4 took 17.87 seconds
Iteration 5 took 16.58 seconds

Average execution time: 19.89 seconds


## Latency Optimization - 3 (Using Triton)

In [5]:
ds_config = {
    "tensor_parallel": {
        "tp_size": 1
    },
    "dtype": torch.float16,
    "replace_with_kernel_inject": True,
    "enable_cuda_graph": True,
    "use_triton": True}

In [21]:
engine = deepspeed.init_inference(
    model=model,
    config=ds_config,
    dtype=torch.float16
)

[2025-03-10 19:49:26,669] [INFO] [logging.py:128:log_dist] [Rank -1] DeepSpeed info: version=0.16.4, git-hash=unknown, git-branch=unknown
[2025-03-10 19:49:26,673] [INFO] [logging.py:128:log_dist] [Rank -1] quantize_bits = 8 mlp_extra_grouping = False, quantize_groups = 1


In [23]:
import time

times = []
for i in range(5):
    start_time = time.time()
    with torch.no_grad():
        generation = engine.module.generate(
            input_ids=input_ids.input_ids,
            attention_mask=input_ids.attention_mask,
            prompt_input_ids=prompt_input_ids.input_ids,
            prompt_attention_mask=prompt_input_ids.attention_mask,
            do_sample=True,
            use_cache=True,
            return_dict_in_generate=True
        )
    end_time = time.time()
    execution_time = end_time - start_time
    times.append(execution_time)
    print(f"Iteration {i+1} took {execution_time:.2f} seconds")

avg_time = sum(times) / len(times)
print(f"\nAverage execution time: {avg_time:.2f} seconds")

Iteration 1 took 17.03 seconds
Iteration 2 took 19.48 seconds
Iteration 3 took 16.12 seconds
Iteration 4 took 21.26 seconds
Iteration 5 took 19.66 seconds

Average execution time: 18.71 seconds


## Latency Optimization - 4 (Using fp16 and triton autotune)

In [8]:
ds_config = {
    "tensor_parallel": {
        "tp_size": 1
    },
    "dtype": torch.float16,
    "replace_with_kernel_inject": True,
    "enable_cuda_graph": True,
    "use_triton": True,
    "triton_autotune": True,
    }

In [9]:
engine = deepspeed.init_inference(
    model=model,
    config=ds_config,
    dtype=torch.float16
)

[2025-03-11 00:27:18,881] [INFO] [logging.py:128:log_dist] [Rank -1] DeepSpeed info: version=0.16.4, git-hash=unknown, git-branch=unknown
[2025-03-11 00:27:18,885] [INFO] [logging.py:128:log_dist] [Rank -1] quantize_bits = 8 mlp_extra_grouping = False, quantize_groups = 1


In [10]:
import time

times = []
for i in range(5):
    start_time = time.time()
    with torch.no_grad():
        generation = engine.module.generate(
            input_ids=input_ids.input_ids,
            attention_mask=input_ids.attention_mask,
            prompt_input_ids=prompt_input_ids.input_ids,
            prompt_attention_mask=prompt_input_ids.attention_mask,
            do_sample=True,
            use_cache=True,
            return_dict_in_generate=True
        )
    end_time = time.time()
    execution_time = end_time - start_time
    times.append(execution_time)
    print(f"Iteration {i+1} took {execution_time:.2f} seconds")

avg_time = sum(times) / len(times)
print(f"\nAverage execution time: {avg_time:.2f} seconds")

Iteration 1 took 15.97 seconds
Iteration 2 took 19.79 seconds
Iteration 3 took 17.64 seconds
Iteration 4 took 16.72 seconds
Iteration 5 took 17.76 seconds

Average execution time: 17.57 seconds


## Others

In [18]:
ds_config = {
        "zero_optimization": {
            "stage": 3,
            "offload_optimizer": {
                "device": "cpu",
                "pin_memory": True
            },
            "overlap_comm": True,
            "reduce_bucket_size": "auto",
            "stage3_prefetch_bucket_size": "auto",
            "stage3_param_persistence_threshold": "auto",
            "stage3_gather_16bit_weights_on_model_save": True
        },
        "train_batch_size": 8,
    }

In [19]:
from transformers.integrations import HfDeepSpeedConfig

In [20]:
hfdsc = HfDeepSpeedConfig(ds_config)

In [21]:
ds_engine = deepspeed.initialize(model=model, config_params=ds_config)[0]
ds_engine.module.eval()

[2025-03-11 00:32:46,427] [INFO] [logging.py:128:log_dist] [Rank 0] DeepSpeed info: version=0.16.4, git-hash=unknown, git-branch=unknown
[2025-03-11 00:32:46,431] [INFO] [config.py:734:__init__] Config mesh_device None world_size = 1
[2025-03-11 00:32:46,471] [INFO] [logging.py:128:log_dist] [Rank 0] DeepSpeed Flops Profiler Enabled: False
[2025-03-11 00:32:46,478] [INFO] [logging.py:128:log_dist] [Rank 0] Creating ZeRO Offload
[2025-03-11 00:32:46,789] [INFO] [utils.py:781:see_memory_usage] DeepSpeedZeRoOffload initialize [begin]
[2025-03-11 00:32:46,791] [INFO] [utils.py:782:see_memory_usage] MA 2.59 GB         Max_MA 2.61 GB         CA 4.2 GB         Max_CA 4 GB 
[2025-03-11 00:32:46,793] [INFO] [utils.py:789:see_memory_usage] CPU Virtual Memory:  used = 39.99 GB, percent = 32.1%
Parameter Offload: Total persistent parameters: 1254898 in 439 params
[2025-03-11 00:32:47,150] [INFO] [utils.py:781:see_memory_usage] DeepSpeedZeRoOffload initialize [end]
[2025-03-11 00:32:47,152] [INFO] 

ParlerTTSForConditionalGeneration(
  (text_encoder): T5EncoderModel(
    (shared): Embedding(32128, 768)
    (encoder): T5Stack(
      (embed_tokens): Embedding(32128, 768)
      (block): ModuleList(
        (0): T5Block(
          (layer): ModuleList(
            (0): T5LayerSelfAttention(
              (SelfAttention): T5Attention(
                (q): Linear(in_features=768, out_features=768, bias=False)
                (k): Linear(in_features=768, out_features=768, bias=False)
                (v): Linear(in_features=768, out_features=768, bias=False)
                (o): Linear(in_features=768, out_features=768, bias=False)
                (relative_attention_bias): Embedding(32, 12)
              )
              (layer_norm): T5LayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (1): T5LayerFF(
              (DenseReluDense): T5DenseGatedActDense(
                (wi_0): Linear(in_features=768, out_features=2048, bias=False)
               

In [22]:
new_model = ds_engine.module

In [23]:
import time

times = []
for i in range(5):
    start_time = time.time()
    # with torch.no_grad():
    generation = new_model.generate(
        input_ids=input_ids.input_ids,
        attention_mask=input_ids.attention_mask,
        prompt_input_ids=prompt_input_ids.input_ids,
        prompt_attention_mask=prompt_input_ids.attention_mask,
        do_sample=True,
        use_cache=True,
        return_dict_in_generate=True)
    end_time = time.time()
    execution_time = end_time - start_time
    times.append(execution_time)
    print(f"Iteration {i+1} took {execution_time:.2f} seconds")

avg_time = sum(times) / len(times)
print(f"\nAverage execution time: {avg_time:.2f} seconds")

RuntimeError: 'weight' must be 2-D